### Obvious things to tweak


Number of layers
Nodes per layer
Dense layer at the end or not

In [15]:
import time
dense_layers = [1]
layer_sizes = [256]
batch_sizes = [32]
conv_layers = [4, 5, 6]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            for batch_size in batch_sizes:
                NAME = "{}conv-{}nodes-{}dense-{}batch-{}".format(conv_layer, layer_size, dense_layer, batch_size, int(time.time()))
                print(NAME)

4conv-256nodes-1dense-32batch-1540834190
5conv-256nodes-1dense-32batch-1540834190
6conv-256nodes-1dense-32batch-1540834190


In [ ]:
# Round two attemps
import tensorflow as tf
import keras
from keras import backend as K

# NOTE: To shrink the VRAM fraction add this to ConfigProto
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
# K.tf.ConfigProto(gpu_options=gpu_options)

cfg = K.tf.ConfigProto(device_count = {'CPU' : 1, 'GPU' : 1})
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.utils import normalize
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator

import pickle
import time

datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.15,
                             zoom_range=0.1, channel_shift_range=10., horizontal_flip=True)

X = pickle.load(open("data/pickled/X.pickle", "rb"))
y = pickle.load(open("data/pickled/y.pickle", "rb"))

# X = keras.utils.normalize(X, axis=1)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, shuffle=False)

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            for batch_size in batch_sizes:
                K.clear_session()
                NAME = "{}conv-{}nodes-{}dense-{}batch-Dropout+BatchReg-{}".format(conv_layer, layer_size, dense_layer, batch_size, int(time.time()))
                print(NAME)
                tensorboard = TensorBoard(log_dir='logs_5/{}'.format(NAME))

                model = Sequential()

                # the 128 can be anything
                # 3,3 is the window of the convolution (3 pixels X 3 pixels)
                # X.shape[1:] is 128, 128, 1 based ont he fact that I transformed the images to 128X128 pixes
                model.add(Conv2D(layer_size, (3,3), activation='relu', input_shape = X.shape[1:]))
                model.add(BatchNormalization())
#                 model.add(Activation("relu"))
#                 model.add(Dropout(0.2))
                model.add(MaxPooling2D(pool_size=(2,2)))

                for l in range(conv_layer-1):
                    model.add(Conv2D(128, (3,3), activation='relu'))
                    model.add(BatchNormalization())
#                     model.add(Activation("relu"))
                    model.add(Dropout(0.1))
                    model.add(MaxPooling2D(pool_size=(2,2)))

                model.add(Flatten())
                for l in range(dense_layer):
                    model.add(Dense(layer_size))
                    model.add(Activation("relu"))
                    model.add(Dropout(0.2))

                # Output Layer
                model.add(Dense(1))
                model.add(Activation('sigmoid'))

                model.compile(loss='binary_crossentropy',
                             optimizer='adam',
                             metrics=['accuracy'])

    #             model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks=[tensorboard])
                model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                                   steps_per_epoch = len(X) / batch_size,
                                   epochs = 40,
                                   validation_data=(X_val, y_val),
                                   callbacks=[tensorboard])

4conv-256nodes-1dense-32batch-Dropout+BatchReg-1540834194
Epoch 1/40
780/779 [==============================] - 198s 254ms/step - loss: 0.5974 - acc: 0.6921 - val_loss: 0.6336 - val_acc: 0.6924
Epoch 2/40
780/779 [==============================] - 194s 249ms/step - loss: 0.4255 - acc: 0.7990 - val_loss: 0.5337 - val_acc: 0.7703
Epoch 3/40
780/779 [==============================] - 194s 249ms/step - loss: 0.3432 - acc: 0.8516 - val_loss: 0.4419 - val_acc: 0.8112
Epoch 4/40
780/779 [==============================] - 195s 250ms/step - loss: 0.2853 - acc: 0.8760 - val_loss: 0.2863 - val_acc: 0.8786
Epoch 5/40
780/779 [==============================] - 194s 249ms/step - loss: 0.2521 - acc: 0.8937 - val_loss: 0.3801 - val_acc: 0.8643
Epoch 6/40
780/779 [==============================] - 199s 256ms/step - loss: 0.2311 - acc: 0.9029 - val_loss: 1.2149 - val_acc: 0.6373
Epoch 7/40
780/779 [==============================] - 195s 250ms/step - loss: 0.2209 - acc: 0.9081 - val_loss: 0.3451 - val_ac

In [5]:
import math
math.ceil(1.25*64)

80

In [6]:
1.25*64

80.0